In [ ]:
import requests
import pandas as pd

In [ ]:
year = [2017, 2018, 2018, 2019, 2020, 2021, 2022, 2023]
url = "https://api.elsevier.com/content/search/scopus/"

In [ ]:
eid_title_cite_list = []
error_list = []

for i in year:
    #First Check For Amount
    params = {
        "query": ("AFFILORG(Chulalongkorn University) AND PUBYEAR = " + str(i)),
        "field": "eid",
        "start": 0
    }
    headers = {
        "Accept": "application/json",
        "X-ELS-APIKey": "fdfa191b9599a981a891f1f6d55bdc95",
    }

    document_count = 0

    response = requests.get(url, headers=headers, params=params)

    if(response.status_code == 200):
        document_count = int(response.json()['search-results']['opensearch:totalResults'])

    start_index = 0
    while (start_index < document_count):
        try:
            params = {
                "query": ("AFFILORG(Chulalongkorn University) AND PUBYEAR = " + str(i)),
                "field": "eid,title,citedby-count",
                "start": start_index
            }

            headers = {
                "Accept": "application/json",
                "X-ELS-APIKey": "fdfa191b9599a981a891f1f6d55bdc95",
            }

            response = requests.get(url, headers=headers, params=params, timeout=30)
            
            if(response.status_code == 200):
                res_json = response.json()
                documents = res_json['search-results']['entry']
                for document in documents:
                    print(i, start_index, document['eid'], document['citedby-count'], document['dc:title'])
                    eid_title_cite_list.append([document['eid'], document['citedby-count'], document['dc:title']])
        except:
            error_list.append([i, start_index])
        start_index += 25





In [ ]:
error_list # <- This Should be length 1 [Record that doesn't have title]

In [ ]:
params = {
    "query": ("AFFILORG(Chulalongkorn University) AND PUBYEAR = 2019"),
    "field": "eid,title,citedby-count",
    "start": 2375
}

headers = {
    "Accept": "application/json",
    "X-ELS-APIKey": "fdfa191b9599a981a891f1f6d55bdc95",
}

response = requests.get(url, headers=headers, params=params, timeout=30)

if(response.status_code == 200):
    res_json = response.json()
    documents = res_json['search-results']['entry']
    for document in documents:
        try:
        # print(i, start_index, document['eid'], document['citedby-count'], document['dc:title'])
            eid_title_cite_list.append([document['eid'], document['citedby-count'], document['dc:title']])
        except:
            pass


In [ ]:
eid_title_cite_df = pd.DataFrame(eid_title_cite_list, columns=['Eid', 'Cite-By_Count', 'Title'])

In [ ]:
eid_title_cite_df.to_csv("UPDATE_CITE_TITLE.csv", index=False)